In [2]:
import os

In [3]:
os.chdir("E:\\INSOFE\\AI and Deep Learning\\DAE_FGen")

In [4]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model

tmp = np.load('cifar_pca_train.npz')
train_data = tmp['data']
train_labels = tmp['labels']

tmp = np.load('cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']

In [6]:
train_data.shape

(50000, 781)

In [7]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 50
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])


In [11]:
history = autoencoder.fit(train_data, train_data,
                    epochs=1,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data),
                    verbose=1)


autoencoder.save('DAE_l1_model.h5') #it has to be loaded everytime this code to be executed freshly
from keras.layers import load_model
autoencoder1 = load_model('DAE_l1_model.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 86s - loss: 0.5296 - mean_squared_error: 0.5296 - val_loss: 0.2092 - val_mean_squared_error: 0.2092


ImportError: cannot import name 'load_model'

In [ ]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)


In [ ]:
input_img1 = Input(shape=(1000,))
crrpt_img1 = Dropout(0.5)(input_img1)
encoded1 = Dense(1000, activation='sigmoid')(crrpt_img1)
decoded1 = Dense(1000, activation='sigmoid')(encoded1)

autoencoder1 = Model(input_img1,decoded1)

autoencoder1.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['binary_crossentropy'])

In [ ]:
history = autoencoder1.fit(htrain_data, htrain_data,
                    epochs=1,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1)


# autoencoder1.save('DAE_l2_model.h5')
from keras.models import load_model
autoencoder1 = load_model('DAE_l2_model.h5')

In [12]:
# Converting labels into one-hot vectors for MLP training

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

In [13]:
# training a simple one hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 25s - loss: 2.1478 - acc: 0.2262 - val_loss: 1.8864 - val_acc: 0.3345
Epoch 2/50
10000/10000 [==============================] - 24s - loss: 1.9246 - acc: 0.3085 - val_loss: 1.7976 - val_acc: 0.3740
Epoch 3/50
 3872/10000 [==========>...................] - ETA: 13s - loss: 1.8074 - acc: 0.3554

KeyboardInterrupt: 

In [ ]:
# Training with unsupervised initialization for layer-1 
# of an MLP using autoencoder weights

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

mlp.layers[1].set_weights(autoencoder.layers[2].get_weights())
mlp.layers[3].set_weights(autoencoder1.layers[2].get_weights())


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))